In [1]:
from sympy import symbols, pi

from robot import Robot, JointType

from math_utils import Vector, Matrix

In [3]:
d1, d2, d3, d4 = symbols('d1 d2 d3 d4')
t1, t2, t3, t4 = symbols('t1 t2 t3 t4')
l1, l2, l3, l4 = symbols('l1 l2 l3 l4')
g = symbols('g')

In [4]:
dh = [
    [0, 0, 0, t1],
    [l1, pi/2, 0, t2],
    [l2,0,0,t3],
    [l3, 0, 0, 0]
]

robot = Robot.from_dh_parameters(dh, '3r')

In [ ]:
# print Transformations
robot.get_transformations()

In [ ]:
# print Rotations
robot.get_rotations()

In [6]:
# end-effector position in zero frame
p_e = (robot.get_end_transformation() @ Vector([0, 0, 0, 1])).simplify()

In [ ]:
# get/ print Z axis in zero fram
axis = robot.get_zero_Z_axis(True)

In [ ]:
# calculate velocities
ws, vs = robot.cal_velocities(True)

In [ ]:
# calculate static kinematics
ws, vs, fs, ns, ts = robot.cal_static_kinematics(True, True)

In [ ]:
# calculate dynamic newton euler kinematics
# define center of mass points for each joint
cof_points = [Vector([0, 0, 0]), Vector([l1 , 0, 0]), Vector([0, 0, 0])]
robot.add_cof(cof_points)
# run calculation of newton euler
stuff = robot.cal_newton_euler(True, True, Vector([0, g, 0]))

In [8]:
# translate vector from specific frame
robot.transform_to_zero_from_i(Vector([0, 0, 0, 1]), 4).simplify()
# rotate vector from specific frame
robot.rotate_to_zero_from_i(Vector([0, 0, 1]), 4).simplify()